# Collecting Typical Blobs based on pre-detected cells from multiple sections

Data to save:
1. Stack, crop range, section, scan information
2. typical blobs
    - props
    - coordinates
    -  
    - 

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import sys, os, time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'cells'))
# sys.path.append(os.path.join(os.environ['REPO_DIR'], 'annotation'))
# sys.path.append(os.path.join(os.environ['REPO_DIR'], 'learning'))

import numpy as np
import scipy
import skimage
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from skimage.measure import regionprops, label, find_contours
from skimage.color import rgb2hsv
from scipy import ndimage as ndi
import pandas
from scipy.signal import argrelmax
from scipy.stats import linregress

from annotation_utilities import *
from registration_utilities import *
from learning_utilities import *
from data_manager import *
from utilities2015 import *
from cell_utilities import *
from metadata import *
from distributed_utilities import download_from_s3

Setting environment for AWS compute node


/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
No vtk


In [2]:
save_folder_path = '/shared/blob_matching_atlas/'
from xj_utilities import *
fun_create_folder(save_folder_path=save_folder_path)

Folder already exists: /shared/blob_matching_atlas/

In [3]:
scan_parameters = {}
scan_parameters['stack'] = 'MD589'
scan_parameters['patch_size'] = 448
scan_parameters['patch_half_size'] = scan_parameters['patch_size']/2
scan_parameters['stride'] = 112
scan_parameters['crop_0_min'] = 7000
scan_parameters['crop_1_min'] = 3000
scan_parameters['crop_0_max'] = 13000
scan_parameters['crop_1_max'] = 14000
scan_parameters['im0max'] = scan_parameters['crop_0_max'] - scan_parameters['crop_0_min']
scan_parameters['im1max'] = scan_parameters['crop_1_max'] - scan_parameters['crop_1_min']
scan_parameters['section_limits'] = metadata_cache['section_limits'][scan_parameters['stack']]
scan_parameters['o_crop'] = True
scan_parameters['o_clear_border'] = True
scan_parameters['o_relabel'] = True
scan_parameters['crop_range_mxmx'] =(scan_parameters['crop_0_min'],scan_parameters['crop_0_max'],scan_parameters['crop_1_min'],scan_parameters['crop_1_max'])
scan_parameters['oriImL1'], scan_parameters['oriImL0'] = metadata_cache['image_shape'][scan_parameters['stack']]
scan_parameters['scan_section_range'] = 1

scan_parameters['prop'] = ['centroid','eccentricity','area','orientation','moments_hu','bbox','equivalent_diameter']
scan_parameters['prop_for_comparison'] = ['area', 'orientation','moments_hu']
scan_parameters['compare_weight'] = [1 for i in range(len(scan_parameters['prop_for_comparison']) + 6)]
scan_parameters['compare_weight'] = np.array(scan_parameters['compare_weight'])/ float(np.sum(scan_parameters['compare_weight']))
scan_parameters['similarity_threshold'] = 0.8
scan_parameters['o_fix_scan_size'] = True
scan_parameters['scan_size'] = 112
scan_parameters['scan_size_coeff'] = 5

In [4]:
tempFp = get_typical_cell_data_filepath(what='scan_parameters',stack='MD589',sec=152)

In [5]:
tempFp

'/shared/blob_matching_atlas/typical_cells/MD589/MD589-N26-2015.07.30-17.44.37_MD589_3_0078/MD589-N26-2015.07.30-17.44.37_MD589_3_0078_ScanParameters.hdf'

In [6]:
save_hdf_v2(scan_parameters,tempFp)

/usr/local/lib/python2.7/dist-packages/pandas/core/generic.py:1299: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->values] [items->None]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [19]:
scan_parameters

{'compare_weight': array([ 0.11111111,  0.11111111,  0.11111111,  0.11111111,  0.11111111,
         0.11111111,  0.11111111,  0.11111111,  0.11111111]),
 'crop_0_max': 13000,
 'crop_0_min': 7000,
 'crop_1_max': 14000,
 'crop_1_min': 3000,
 'crop_range_mxmx': (7000, 13000, 3000, 14000),
 'im0max': 6000,
 'im1max': 11000,
 'o_clear_border': True,
 'o_crop': True,
 'o_fix_scan_size': True,
 'o_relabel': True,
 'oriImL0': 11936,
 'oriImL1': 15520,
 'patch_half_size': 224,
 'patch_size': 448,
 'prop': ['centroid',
  'eccentricity',
  'area',
  'orientation',
  'moments_hu',
  'bbox',
  'equivalent_diameter'],
 'prop_for_comparison': ['area', 'orientation', 'moments_hu'],
 'scan_section_range': 1,
 'scan_size': 112,
 'scan_size_coeff': 5,
 'section_limits': array([ 92, 370]),
 'similarity_threshold': 0.8,
 'stack': 'MD589',
 'stride': 112}

In [7]:
patch_size = scan_parameters['patch_size']
patch_half_size = scan_parameters['patch_half_size']
stride = scan_parameters['stride']
# Brain stem
crop_0_min = scan_parameters['crop_0_min']
crop_0_max = scan_parameters['crop_1_min']
crop_1_min = scan_parameters['crop_0_max']
crop_1_max = scan_parameters['crop_1_max']
# Inside
# crop_0_min = 4000
# crop_0_max = 6000
# crop_1_min = 2400
# crop_1_max = 4400
# Whole
# crop_0_min = 0
# crop_1_min = 0
# _, _, crop_1_max, crop_0_max = get_default_gridspec(stack=stack, patch_size=patch_size, stride=stride)
im0max = crop_0_max - crop_0_min
im1max = crop_1_max - crop_1_min

# Options
option_rescale_grayscale = True
option_THimage_save = True;
option_save_image_extension = '.png'

# Process images
option_label_connectivity = 4;

### Load and crop origninal images

In [8]:
# Functionalize loading data
img_filename = {}
stack = scan_parameters['stack']
section_begin, section_end = scan_parameters['section_limits']
secList = range(section_begin, section_end+1)
# Download all the data for MD589
for tempSec in secList:
    try:
        img_filename[tempSec] = DataManager.get_image_filepath(stack=stack, section=tempSec, resol='lossless', version='cropped')
    except:
        sys.stderr.write('Invalid section %d...\n'%tempSec)
        secList.remove(tempSec)
        continue
    if os.path.isfile(img_filename[tempSec]):
        sys.stderr.write('Image file is already available. \n')
        continue
    else:
        try:
            download_from_s3(img_filename[tempSec])
        except:
            sys.stderr.write('Download fail. Skip this file for section %d...\n'%tempSec)
            secList.remove(tempSec)
            continue   
scan_parameters['secList'] = secList

Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Invalid section 118...
Image file is already available. 
Image file is already available. 
Image file is already ava

In [9]:
from multiprocess import Pool

In [10]:
result = Pool(8)
result.map(lambda sec: fun_load_data_collect_typical_blobs(sec=sec, scan_parameters=scan_parameters), secList[0:31])

Scanning section 92
Warrning: Scaned section(s) not included in input im_blob_prop
Section 92 Finished percentage: 0.000000
Section 92 Finished percentage: 19.451469
Scanning section 93
Section 93 Finished percentage: 0.000000
Scanning section 95
Scanning section 97
Scanning section 98
Section 95 Finished percentage: 0.000000
Scanning section 96
Scanning section 99
Scanning section 94
Section 97 Finished percentage: 0.000000
Section 98 Finished percentage: 0.000000
Section 92 Finished percentage: 38.902937
Section 96 Finished percentage: 0.000000
Section 94 Finished percentage: 0.000000
Section 99 Finished percentage: 0.000000
Section 92 Finished percentage: 58.354406
Section 93 Finished percentage: 21.299255
Section 95 Finished percentage: 21.190930
Section 97 Finished percentage: 18.796992
Section 98 Finished percentage: 17.096940
Section 99 Finished percentage: 18.152115
Section 96 Finished percentage: 17.711654
Section 94 Finished percentage: 18.789929
Section 92 Finished percentag

Section 113 Finished percentage: 93.997583
Section 119 Finished percentage: 14.749263
Section 117 Finished percentage: 29.846292
Scanning section 120
Section 120 Finished percentage: 0.000000
Section 116 Finished percentage: 75.517293
Section 119 Finished percentage: 29.498525
Scanning section 121
Section 117 Finished percentage: 44.769437
Section 121 Finished percentage: 0.000000
Scanning section 122
Section 122 Finished percentage: 0.000000
Section 119 Finished percentage: 44.247788
Section 117 Finished percentage: 59.692583
Scanning section 123
Section 123 Finished percentage: 0.000000
Section 116 Finished percentage: 90.620752
Section 119 Finished percentage: 58.997050
Section 121 Finished percentage: 18.439978
Section 120 Finished percentage: 18.768769
Section 117 Finished percentage: 74.615729
Section 119 Finished percentage: 73.746313
Section 122 Finished percentage: 21.574973
Section 117 Finished percentage: 89.538875
Section 119 Finished percentage: 88.495575
Section 123 Finis

[([[92,
    58,
   [92,
    81,
   [92,
    88,
   [92,
    101,
   [92,
    128,
   [92,
    152,
   [92,
    159,
   [92,
    178,
   [92,
    193,
   [92,
    201,
   [92,
    251,
   [92,
    267,
   [92,
    272,
   [92,
    306,
   [92,
    309,
   [92,
    314,
   [92,
    421,
   [92,
    443,
   [92,
    452,
   [92,
    473,
   [92,
    493,
   [92,
    498,
   [92,
    508,
   [92,
    523,
   [92,
    532,
   [92,
    593,
   [92,
    631,
   [92,
    668,
   [92,
    700,
   [92,
    711,
   [92,
    714,
   [92,
    744,
   [92,
    747,
   [92,
    766,
   [92,
    833,
   [92,
    867,
   [92,
    872,
   [92,
    905,
   [92,
    916,
   [92,
    938,
   [92,
    998,
   [92,
    1011,
   [92,
    1031,
   [92,
    1033,
   [92,
    1050,
   [92,
    1055,
   [92,
    1079,
   [92,
    1091,
   [92,
    1100,
   [92,
    1147,
   [92,
    1152,
   [92,
    1166,
   [92,
    1180,
   [92,
    1189,
   [92,
    1195,
   [92,
    1196,
   [92,
    1205,
   [92,
    1207,


In [19]:
sec = 152

In [21]:
typical_blobs = fun_load_data_collect_typical_blobs(sec=sec, scan_parameters=scan_parameters)

Scanning section 152
Section 152 Finished percentage: 0.000000
Section 152 Finished percentage: 5.701254
Section 152 Finished percentage: 11.402509
Section 152 Finished percentage: 17.103763
Section 152 Finished percentage: 22.805017
Section 152 Finished percentage: 28.506271
Section 152 Finished percentage: 34.207526
Section 152 Finished percentage: 39.908780
Section 152 Finished percentage: 45.610034
Section 152 Finished percentage: 51.311288
Section 152 Finished percentage: 57.012543
Section 152 Finished percentage: 62.713797
Section 152 Finished percentage: 68.415051
Section 152 Finished percentage: 74.116306
Section 152 Finished percentage: 79.817560
Section 152 Finished percentage: 85.518814
Section 152 Finished percentage: 91.220068
Section 152 Finished percentage: 96.921323


In [3]:
metadata_cache['sections_to_filenames']['MD589'][1]

'MD589-N1-2015.07.30-16.19.59_MD589_1_0001'

In [27]:
a = typical_blobs[0][0]

In [45]:
save_prop = list(scan_parameters['prop'])
save_prop.append('coords')

In [46]:
store_dic = {}
for prop in save_prop:
    store_dic[prop] = a[2][prop]

In [12]:
result